In [1]:
import boa

In [2]:
admin = boa.env.generate_address()
fee_receiver = boa.env.generate_address()
trader = boa.env.generate_address()
print(admin, fee_receiver, trader)

0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E 0xA73d7cddCf77c00827459f986bf828999B58C6Fe 0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B


In [3]:
coin_a = boa.load("../../contracts/twocrypto-ng/mocks/ERC20Mock.vy", "coin_a", "coin_a", 18)
coin_b = boa.load("../../contracts/twocrypto-ng/mocks/ERC20Mock.vy", "coin_b", "coin_b", 18)


# we mint the trade 1000 coins each (10^21)
coin_a._mint_for_testing(trader, 10**21, sender=trader)
coin_b._mint_for_testing(trader, 10**21, sender=trader)
print(coin_a, coin_b)

/Users/mo/Documents/CurveDocs/venv-notebook/lib/python3.11/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")


<../../contracts/twocrypto-ng/mocks/ERC20Mock.vy at 0x0880cf17Bd263d3d3a5c09D2D86cCecA3CcbD97c, compiled with vyper-0.3.10+9136169>
<storage: name=coin_a                          , symbol=coin_a                          , decimals=18, balanceOf={'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': 1000000000000000000000}, allowances={}, totalSupply=1000000000000000000000> <../../contracts/twocrypto-ng/mocks/ERC20Mock.vy at 0x2cb6bCe32aeF4eD506382896e702DE7Ff109D9E9, compiled with vyper-0.3.10+9136169>
<storage: name=coin_b                          , symbol=coin_b                          , decimals=18, balanceOf={'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': 1000000000000000000000}, allowances={}, totalSupply=1000000000000000000000>


In [4]:
pool_implementation = boa.load_partial("../../contracts/twocrypto-ng/main/CurveTwocryptoOptimized.vy")
gauge_implementation = boa.load_partial("../../contracts/twocrypto-ng/main/LiquidityGauge.vy")
math_implementation = boa.load("../../contracts/twocrypto-ng/main/CurveCryptoMathOptimized2.vy")
views_implementation = boa.load("../../contracts/twocrypto-ng/main/CurveCryptoViews2Optimized.vy")

with boa.env.prank(admin):
    factory = boa.load("../../contracts/twocrypto-ng/main/CurveTwocryptoFactory.vy")
    factory.initialise_ownership(fee_receiver, admin)

    factory.set_pool_implementation(pool_implementation.deploy_as_blueprint().address, 0)
    factory.set_gauge_implementation(gauge_implementation.deploy_as_blueprint().address)
    factory.set_math_implementation(math_implementation.address)
    factory.set_views_implementation(views_implementation.address)

print(
    "Factory: " + factory.address + "\n" +
    "Pool Implementation: " + factory.pool_implementations(0) + "\n" +
    "Gauge Implementation: " + factory.gauge_implementation() + "\n" +
    "Math Implementation: " + factory.math_implementation() + "\n" +
    "Views Implementation: " + factory.views_implementation()
    )

/Users/mo/Documents/CurveDocs/venv-notebook/lib/python3.11/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


Factory: 0xC6Acb7D16D51f72eAA659668F30A40d87E2E0551
Pool Implementation: 0x3d06E92f20305D9a2D71a1D479E9EE22690Ae7E4
Gauge Implementation: 0x78548820b365886d05009F1127bf553603E5A836
Math Implementation: 0xB822167C7EefF0B53DcfDEE2D8fe73dEDB25505b
Views Implementation: 0xDa5e407C7b1887E7f76c920B70614e73feA0DDA1


In [5]:
# prepare pool params for deployment

A = 400000
gamma = 145000000000000
mid_fee = 26000000
out_fee = 45000000
allowed_extra_profit = 2000000000000
fee_gamma = 230000000000000
adjustment_step = 146000000000000
ma_time = 600
xcp_ma_time = 1800 * 24
initial_price = 10**18  # 1:1 at the start
admin_fee = 5 * 10**9


# deploying the pool

pool = factory.deploy_pool(
    "ng", "ng", [coin_a.address, coin_b.address],
    0, A, gamma, mid_fee, out_fee, fee_gamma, allowed_extra_profit, adjustment_step, int(ma_time/0.693), initial_price
)
pool = pool_implementation.at(pool)
pool

<../../contracts/twocrypto-ng/main/CurveTwocryptoOptimized.vy at 0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2, compiled with vyper-0.3.10+9136169>
<storage: cached_price_scale=1000000000000000000, cached_price_oracle=1000000000000000000, cached_xcp_oracle=0, last_prices=1000000000000000000, last_timestamp=580322366264719322237204769971829526793105608206, last_xcp=0, xcp_ma_time=62324, initial_A_gamma=136112946768375385385349842972852284582400000, initial_A_gamma_time=0, future_A_gamma=136112946768375385385349842972852284582400000, future_A_gamma_time=0, balances=[0, 0], D=0, xcp_profit=0, xcp_profit_a=1000000000000000000, virtual_price=0, packed_rebalancing_params=680564733841876929619973849625130958848000000000865, packed_fee_params=8847341539944400050877843276543133320576000000, last_admin_fee_claim_timestamp=0, admin_lp_virtual_balance=0, balanceOf={}, allowance={}, totalSupply=0, nonces={}>

## Adding Liquidity to the Pool

First, we define a function to check the balances of the trader and the pool:

In [6]:
def get_balances():
    trader_balance_a = coin_a.balanceOf(trader)
    trader_balance_b = coin_b.balanceOf(trader)
    lp_tokens = pool.balanceOf(trader)
    pool_balance_a = pool.balances(0)
    pool_balance_b = pool.balances(1)

    print(
        "TRADER BALANCES:\n"
        "coin a: " + str(trader_balance_a / 10**18) + "\n"
        "coin b: " + str(trader_balance_b / 10**18) + "\n"
        "lp tokens: " + str(lp_tokens / 10**18) + "\n" +
        "---------\n"
        "POOL BALANCES:\n"
        "coin a: " + str(pool_balance_a / 10**18) + "\n"
        "coin b: " + str(pool_balance_b / 10**18)
    )

get_balances()

TRADER BALANCES:
coin a: 1000.0
coin b: 1000.0
lp tokens: 0.0
---------
POOL BALANCES:
coin a: 0.0
coin b: 0.0


In [7]:
# grant max approval

with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        coin.approve(pool, 2**256-1)

print(coin_a._storage.allowances.get())
print(coin_b._storage.allowances.get())

{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}
{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}


---

The **`add_liquidity`** function uses the following input values:

| Input            | Type                | Description                                           |
| ---------------- | ------------------- | ----------------------------------------------------- |
| `amounts`        | `uint256[N_COINS]`  | Amount of each coin to add. `amount_a` is for `Pool.coins(0)`, `amount_b` `Pool.coins(1)`.                           |
| `min_mint_amount`| `uint256`           | Minimum amount of LP tokens to mint.                  |
| `receiver`       | `address`           | Receiver of the LP tokens; defaults to `msg.sender`.  |

First, we add some liquidity single sided and check the coin balances before and after:

In [8]:
amounts_a = 10**20  # Pool.coins(0)
amounts_b = 10**20  # Pool.coins(1)
min_mint_amount = 99
receiver = trader

# we check the balances before providing liquidity
get_balances()

with boa.env.prank(trader):
    pool.add_liquidity([amounts_a, amounts_b], min_mint_amount, receiver)

# we check again after adding liquidity
print("-----------")
get_balances()

TRADER BALANCES:
coin a: 1000.0
coin b: 1000.0
lp tokens: 0.0
---------
POOL BALANCES:
coin a: 0.0
coin b: 0.0
-----------
TRADER BALANCES:
coin a: 900.0
coin b: 900.0
lp tokens: 100.0
---------
POOL BALANCES:
coin a: 100.0
coin b: 100.0


As we can see, we added 100 token each coin. In exchange we received 100 LP tokens.

Now lets add liquidity again, but this time single-sided: 10 coin_a and 100 coin_b.

In [9]:
amounts_a = 10**19  # Pool.coins(0)
amounts_b = 10**20  # Pool.coins(1)
min_mint_amount = 0
receiver = trader

# we check the balances before providing liquidity
get_balances()

with boa.env.prank(trader):
    pool.add_liquidity([amounts_a, amounts_b], min_mint_amount, receiver)

# we check again after adding liquidity
print("-----------")
get_balances()

TRADER BALANCES:
coin a: 900.0
coin b: 900.0
lp tokens: 100.0
---------
POOL BALANCES:
coin a: 100.0
coin b: 100.0
-----------
TRADER BALANCES:
coin a: 890.0
coin b: 800.0
lp tokens: 148.47325226409856
---------
POOL BALANCES:
coin a: 110.0
coin b: 200.0


---

**`calc_token_fee`**

This function allows to calculate the fee charged at a certain state of the pool when adding liquidity.

`Pool.calc_token_fee(amounts: uint256[N_COINS], xp: uint256[N_COINS]) -> uint256:`

| Input            | Type                | Description                                           |
| ---------------- | ------------------- | ----------------------------------------------------- |
| `amounts`        | `uint256[N_COINS]`  | Amount of each coin to add. `amount_a` is for `Pool.coins(0)`, `amount_b` `Pool.coins(1)`. |
| `xp`| `uint256[N_COINS]`               | Pool balances multiplied by the coin precisions. todo: any by `price_scale?`      |


Precisions are needed to a precise calculations. If a coin has 18 decimals, its precisions will be 1. For 17 decimals its 10. For 16 decimals its 100. Etc...

Minimum fee when adding liquidity is 100000, this occurs when the pool is perfectly balanced and liquidity is added in a balanced portion?

In [10]:
amounts_a = 10**19
amounts_b = 10**20
precision_a = 1             # to use the deployed pools variable, do pool.precisions()[0]
precision_b = 1             # to use the deployed pools variable, do pool.precisions()[1]
pool_balance_a = pool.balances(0)          # to use the deployed pools variable, do pool.balances(0)
pool_balance_b = pool.balances(1)          # to use the deployed pools variable, do pool.balances(1)
xp_a = pool_balance_a * precision_a
xp_b = pool_balance_b * precision_b

pool.calc_token_fee([amounts_a, amounts_b], [xp_a, xp_b])

18487938

One might try to set `pool_balance_a` and/or `pool_balance_b` to 0 to calculate the fee when adding liquidity to an empty pool: This does not work (dividing with 0 is not possible) for the sole reason that when adding liquidity, the fee is calculated based on the pool balances AFTER the tokens have been transfered into the pool.

---

Lets check the fees. We need to call `remove_liquidity_one_coin` as this function calls the internal `_claim_admin_fees` function. The fees that occur for removing single sided is not being cosidered in the fee claiming, as the fee claiming is the first "action" in the function.

In [11]:
print(coin_a.balanceOf(fee_receiver) / 10**18)
print(coin_b.balanceOf(fee_receiver) / 10**18)

with boa.env.prank(trader):
    pool.remove_liquidity_one_coin(10**18, 0, 0, trader)

print(coin_a.balanceOf(fee_receiver) / 10**18)
print(coin_b.balanceOf(fee_receiver) / 10**18)


0.0
0.0
0.0498583042932385
0.09065146235134273
